


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [2]:
# Load the data and preprocess data and store corpus in raw_text
filename = '/content/corpus.txt'
raw_text = open(filename,'r',encoding='utf-8').read()
#raw_text = raw_text.lower()

In [3]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [4]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([raw_text])
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'the': 2, '”': 3, 'and': 4, 'to': 5, 'a': 6, 'she': 7, 'of': 8, 'it': 9, 'said': 10, 'alice': 11, 'in': 12, 'was': 13, 'you': 14, 'i': 15, 'that': 16, 'as': 17, 'her': 18, 'at': 19, 'on': 20, 'with': 21, 'had': 22, 'all': 23, 'be': 24, 'for': 25, 'so': 26, 'very': 27, 'not': 28, 'this': 29, 'little': 30, 'but': 31, '“i': 32, 'they': 33, 'out': 34, 'he': 35, 'down': 36, 'what': 37, 'up': 38, 'is': 39, 'one': 40, 'his': 41, 'about': 42, 'were': 43, 'like': 44, 'went': 45, 'herself': 46, 'them': 47, 'again': 48, 'know': 49, 'then': 50, 'would': 51, 'could': 52, 'have': 53, 'no': 54, 'thought': 55, 'when': 56, 'if': 57, 'do': 58, 'time': 59, 'or': 60, 'there': 61, 'queen': 62, 'into': 63, 'me': 64, 'see': 65, 'off': 66, 'king': 67, 'your': 68, 'did': 69, 'began': 70, 'its': 71, 'by': 72, 'an': 73, 'my': 74, 'mock': 75, 'turtle': 76, '“and': 77, 'quite': 78, 'hatter': 79, 'gryphon': 80, 'way': 81, 'who': 82, 'don’t': 83, 'their': 84, 'think': 85, 'much': 86, 'say': 87, 'some': 

In [5]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([raw_text])[0]
print(tokens)

[314, 15, 36, 2, 95, 729, 11, 13, 288, 5, 106, 27, 493, 8, 368, 72, 18, 444, 20, 2, 1131, 4, 8, 369, 142, 5, 58, 143, 60, 730, 7, 22, 876, 63, 2, 346, 18, 444, 13, 877, 31, 9, 22, 54, 731, 60, 1132, 12, 9, 77, 37, 39, 2, 217, 8, 6, 346, 3, 55, 11, 1583, 731, 60, 1132, 3, 26, 7, 13, 878, 12, 18, 370, 371, 17, 128, 17, 7, 52, 25, 2, 494, 154, 144, 18, 445, 27, 629, 4, 879, 347, 2, 1133, 8, 446, 6, 1584, 1585, 51, 24, 732, 2, 561, 8, 191, 38, 4, 1134, 2, 1586, 56, 289, 6, 145, 95, 21, 1587, 155, 239, 290, 72, 18, 61, 13, 142, 26, 27, 1135, 12, 16, 1136, 69, 11, 85, 9, 26, 27, 86, 34, 8, 2, 81, 5, 269, 2, 95, 87, 5, 270, 182, 156, 254, 156, 15, 183, 24, 562, 3, 56, 7, 55, 9, 112, 1137, 9, 1138, 5, 18, 16, 7, 271, 5, 53, 1588, 19, 29, 31, 19, 2, 59, 9, 23, 162, 78, 733, 31, 56, 2, 95, 1589, 177, 6, 447, 34, 8, 71, 1139, 495, 4, 100, 19, 9, 4, 50, 348, 20, 11, 1140, 5, 18, 208, 25, 9, 1590, 630, 18, 371, 16, 7, 22, 107, 121, 272, 6, 95, 21, 406, 6, 1139, 495, 60, 6, 447, 5, 209, 34, 8, 9, 4,

In [6]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  27556


In [7]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)
print(np.shape(X))

(27556, 50)


In [8]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 3000)              387000    
Total params: 771,480
Trainable params: 771,480
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=4, batch_size=128, validation_split=0.2 ) 

Epoch 1/4
173/173 [==============================] - 31s 178ms/step - loss: 6.5061 - accuracy: 0.0517 - val_loss: 6.2582 - val_accuracy: 0.0796
Epoch 2/4
173/173 [==============================] - 30s 175ms/step - loss: 6.0265 - accuracy: 0.0531 - val_loss: 6.2038 - val_accuracy: 0.0796
Epoch 3/4
173/173 [==============================] - 30s 173ms/step - loss: 5.9113 - accuracy: 0.0569 - val_loss: 6.2044 - val_accuracy: 0.0867
Epoch 4/4
173/173 [==============================] - 30s 172ms/step - loss: 5.7975 - accuracy: 0.0649 - val_loss: 6.1833 - val_accuracy: 0.0967


In [10]:
reverse_word_map = {v: k for (k, v) in word_index.items()}
print(reverse_word_map)## Create word to idx map using tokenizer.word_index

{1: '<OOV>', 2: 'the', 3: '”', 4: 'and', 5: 'to', 6: 'a', 7: 'she', 8: 'of', 9: 'it', 10: 'said', 11: 'alice', 12: 'in', 13: 'was', 14: 'you', 15: 'i', 16: 'that', 17: 'as', 18: 'her', 19: 'at', 20: 'on', 21: 'with', 22: 'had', 23: 'all', 24: 'be', 25: 'for', 26: 'so', 27: 'very', 28: 'not', 29: 'this', 30: 'little', 31: 'but', 32: '“i', 33: 'they', 34: 'out', 35: 'he', 36: 'down', 37: 'what', 38: 'up', 39: 'is', 40: 'one', 41: 'his', 42: 'about', 43: 'were', 44: 'like', 45: 'went', 46: 'herself', 47: 'them', 48: 'again', 49: 'know', 50: 'then', 51: 'would', 52: 'could', 53: 'have', 54: 'no', 55: 'thought', 56: 'when', 57: 'if', 58: 'do', 59: 'time', 60: 'or', 61: 'there', 62: 'queen', 63: 'into', 64: 'me', 65: 'see', 66: 'off', 67: 'king', 68: 'your', 69: 'did', 70: 'began', 71: 'its', 72: 'by', 73: 'an', 74: 'my', 75: 'mock', 76: 'turtle', 77: '“and', 78: 'quite', 79: 'hatter', 80: 'gryphon', 81: 'way', 82: 'who', 83: 'don’t', 84: 'their', 85: 'think', 86: 'much', 87: 'say', 88: 'som

In [11]:
# Complete the code to return next n words greedily
def next_tokens(input_str,n):
  print("Seed: \n", input_str)
  #x = numpy.array(tokenizer.texts_to_sequences([input_str])[0])
  final_string = []
  for i in range(n):
    x = numpy.array(tokenizer.texts_to_sequences([input_str])[0])[numpy.newaxis,:]
    prediction = model.predict(x[:,-50:] , verbose=0)
    idx = numpy.argmax(prediction)
    wrd = reverse_word_map[idx]
    input_str = input_str + ' ' +wrd
    #print(x)
    final_string.append(wrd)
  # get next word index. Use reverse_word_map to get the word
  final_string = ' '.join(final_string)
  return final_string


In [12]:
# pick a random seed

start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ' '.join([reverse_word_map[value] for value in pattern])

print(next_tokens( input_str , 10))

Seed: 
 the moral of that is—‘be what you would seem to be’—or if you’d like it put more simply—‘never imagine yourself not to be otherwise than what it might appear to others that what you were or might have been was not otherwise than what you had been would have appeared
to the little little little little little little little little


In [13]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."
inp_idx = tokenizer.texts_to_sequences([input_str])[0]
inp = ' '.join([reverse_word_map[value] for value in inp_idx[:50]])

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
print(next_tokens( inp, 10))

Seed: 
 the boy laughed at the fright he had caused this time the <OOV> left angrily the <OOV> day as the boy went up the small <OOV> he suddenly saw a <OOV> <OOV> his <OOV> he cried as hard as he could <OOV> <OOV> <OOV> ” but not a <OOV> <OOV>
” the little little little little little little little little


# Character based LSTM Model 1

In [14]:
# User the preprocess data and create raw_text
filename = '/content/corpus.txt'
raw_text = open(filename,'r',encoding='utf-8').read()
#raw_text = raw_text.lower()
# create mapping of unique characters to integers
chars = sorted(list(set(raw_text)))

char_to_int = {chars[i]: i for i in range(len(chars))}
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '(': 3, ')': 4, ',': 5, '-': 6, '.': 7, ':': 8, ';': 9, '?': 10, 'A': 11, 'B': 12, 'C': 13, 'D': 14, 'E': 15, 'F': 16, 'G': 17, 'H': 18, 'I': 19, 'J': 20, 'K': 21, 'L': 22, 'M': 23, 'N': 24, 'O': 25, 'P': 26, 'Q': 27, 'R': 28, 'S': 29, 'T': 30, 'U': 31, 'V': 32, 'W': 33, 'X': 34, 'Y': 35, 'Z': 36, '[': 37, ']': 38, '_': 39, 'a': 40, 'b': 41, 'c': 42, 'd': 43, 'e': 44, 'f': 45, 'g': 46, 'h': 47, 'i': 48, 'j': 49, 'k': 50, 'l': 51, 'm': 52, 'n': 53, 'o': 54, 'p': 55, 'q': 56, 'r': 57, 's': 58, 't': 59, 'u': 60, 'v': 61, 'w': 62, 'x': 63, 'y': 64, 'z': 65, 'ù': 66, '—': 67, '‘': 68, '’': 69, '“': 70, '”': 71}


In [15]:
# Print the total characters and character vacob size
n_chars = len(raw_text)
n_vocab = len(chars)
print(n_chars)
print(n_vocab)

142477
72


In [16]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  input = raw_text[i:i + seq_length]
  target = raw_text[i + seq_length]
  #if char_to_int[target]==1 or char_to_int[target]==0: #Skip samples where target word is OOV
    #continue
  dataX.append([char_to_int[input[i]] for i in range(seq_length)])
  dataY.append(char_to_int[target])

  # Write code here



n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
print(dataY)
print(dataX[:5])

Total Patterns:  142377
[44, 1, 41, 40, 53, 50, 5, 1, 40, 53, 43, 1, 54, 45, 1, 47, 40, 61, 48, 53, 46, 1, 53, 54, 59, 47, 48, 53, 46, 1, 59, 54, 1, 43, 54, 8, 1, 54, 53, 42, 44, 1, 54, 57, 1, 59, 62, 48, 42, 44, 1, 58, 47, 44, 1, 47, 40, 43, 1, 55, 44, 44, 55, 44, 43, 1, 48, 53, 59, 54, 1, 59, 47, 44, 1, 41, 54, 54, 50, 1, 47, 44, 57, 1, 58, 48, 58, 59, 44, 57, 1, 62, 40, 58, 1, 57, 44, 40, 43, 48, 53, 46, 5, 1, 41, 60, 59, 1, 48, 59, 1, 47, 40, 43, 1, 53, 54, 1, 55, 48, 42, 59, 60, 57, 44, 58, 1, 54, 57, 1, 42, 54, 53, 61, 44, 57, 58, 40, 59, 48, 54, 53, 58, 1, 48, 53, 1, 48, 59, 5, 1, 70, 40, 53, 43, 1, 62, 47, 40, 59, 1, 48, 58, 1, 59, 47, 44, 1, 60, 58, 44, 1, 54, 45, 1, 40, 1, 41, 54, 54, 50, 5, 71, 1, 59, 47, 54, 60, 46, 47, 59, 1, 11, 51, 48, 42, 44, 1, 70, 62, 48, 59, 47, 54, 60, 59, 1, 55, 48, 42, 59, 60, 57, 44, 58, 1, 54, 57, 1, 42, 54, 53, 61, 44, 57, 58, 40, 59, 48, 54, 53, 58, 10, 71, 1, 29, 54, 1, 58, 47, 44, 1, 62, 40, 58, 1, 42, 54, 53, 58, 48, 43, 44, 57, 48, 53, 46,

In [17]:
# reshape X to be [samples, time steps, features]
X = numpy.array(dataX)
print(np.shape(X))

# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)
print(y[1])

(142377, 100)
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [18]:
embedding_dim =100
max_length =100

In [19]:
model = Sequential()
model.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          7200      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 72)                18504     
Total params: 391,272
Trainable params: 391,272
Non-trainable params: 0
_________________________________________________________________


In [20]:
#model.fit(X, y, epochs=20, batch_size=128)
model.fit(X, y, epochs=10, batch_size=128)
#model.save()

Epoch 1/10
1113/1113 [==============================] - 917s 823ms/step - loss: 2.3335
Epoch 2/10
1113/1113 [==============================] - 920s 826ms/step - loss: 1.8164
Epoch 3/10
1113/1113 [==============================] - 898s 807ms/step - loss: 1.6134
Epoch 4/10
1113/1113 [==============================] - 900s 809ms/step - loss: 1.4876
Epoch 5/10
1113/1113 [==============================] - 908s 816ms/step - loss: 1.3990
Epoch 6/10
1113/1113 [==============================] - 910s 817ms/step - loss: 1.3328
Epoch 7/10
1113/1113 [==============================] - 891s 801ms/step - loss: 1.2798
Epoch 8/10
1113/1113 [==============================] - 896s 805ms/step - loss: 1.2307
Epoch 9/10
1113/1113 [==============================] - 901s 810ms/step - loss: 1.1907
Epoch 10/10
1113/1113 [==============================] - 918s 825ms/step - loss: 1.1572


In [21]:
#implement mapping of integer to character
int_to_char = {v: k for (k, v) in char_to_int.items()}

In [22]:
'''
Complete code below to get the generated string using the model.
'''

def predict_next_100_chars(pattern,x):
  print("Seed: \n", pattern)
  #p = numpy.array([char_to_int[pattern[i]] for i in range(len(pattern))])[numpy.newaxis,:]
  final_string = []
  for i in range(x):
    # Complete Code
    p = numpy.array([char_to_int[pattern[i]] for i in range(len(pattern))])[numpy.newaxis,:]
    prediction = model.predict(p[:,-100:] , verbose=0)
    idx = numpy.argmax(prediction)
    wrd = int_to_char[idx]
    pattern += wrd
    final_string.append(wrd)
  final_string = ''.join(final_string)
  return final_string



In [23]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_chars(input_str,200))

Seed: 
  in one hand and a piece of bread-and-butter in the other.
“I beg pardon, your Majesty,” he began, “
and that’s the sea.” “I don’t think I can’t be seen that is the sea.” “I don’t think I can’t be seen that is the sea.” “I don’t think I can’t be seen that is the sea.” “I don’t think I can’t be seen t


In [25]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.

 
print(predict_next_100_chars(input_str[:100],200))

Seed: 
 The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, a
nd the poor little thing that she was no one of the sea.
“I don’t be a great distance,” said the Cat.
“I don’t think I can’t be seen that is the sea.” “I don’t think I can’t be seen that is the sea.” 


## Character based LSTM Model 2


In [26]:
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
model1.fit(X, y, epochs=10, batch_size=64)

Epoch 1/10
2225/2225 [==============================] - 2680s 1s/step - loss: 2.2356
Epoch 2/10
2225/2225 [==============================] - 2823s 1s/step - loss: 1.6671
Epoch 3/10
2225/2225 [==============================] - 2848s 1s/step - loss: 1.4833
Epoch 4/10
2225/2225 [==============================] - 2786s 1s/step - loss: 1.3810
Epoch 5/10
2225/2225 [==============================] - 2778s 1s/step - loss: 1.3082
Epoch 6/10
2225/2225 [==============================] - 2764s 1s/step - loss: 1.2534
Epoch 7/10
2225/2225 [==============================] - 2800s 1s/step - loss: 1.2053
Epoch 8/10
  46/2225 [..............................] - ETA: 44:43 - loss: 1.1320

KeyboardInterrupt: ignored

In [30]:
# Generate the sequence similar to above methods

'''
Complete code below to get the generated string using the model.
'''

def predict_next_100_chars(pattern,x):
  print("Seed: \n", pattern)
  #p = numpy.array([char_to_int[pattern[i]] for i in range(len(pattern))])[numpy.newaxis,:]
  final_string = []
  for i in range(x):
    # Complete Code
    p = numpy.array([char_to_int[pattern[i]] for i in range(len(pattern))])[numpy.newaxis,:]
    prediction = model1.predict(p[:,-100:] , verbose=0)
    idx = numpy.argmax(prediction)
    wrd = int_to_char[idx]
    pattern += wrd
    final_string.append(wrd)
  final_string = ''.join(final_string)
  return final_string



In [31]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_chars(input_str,200))

Seed: 
  “Yes, I think you’d better leave off,” said the Gryphon: and Alice was only too glad to do so.
“Sha
ll the trees to herself, and the pabbit was a little thing as she was a little thing as she was a little thing as she was a little thing as she was a little thing as she was a little thing as she was 


In [32]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.

 
print(predict_next_100_chars(input_str[:100],200))

Seed: 
 The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, a
nd the pool little same side of the same to the same to herself of the same to the same to herself, and the pool little said to herself, “I shall be a little golden the table, and the pabbit was a lit


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:** Performance on unseen data was slightly bad than train data as it is totally unaware of the unseen data and tries to predict next sequence based on the pattern of train data by which the model is fitted . I guess the performance could be improved more if the model was trained for the 20 epochs as specified in the code .

**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:** Model2 is better in capturing the relationship between the words as well as between the characters than model1 although the model2 is trained for smaller number of epochs than model1 . As a result model2 is able to predict more meaningful word sequence or sentences than that predicted by model1 .

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 4.3MB/s 
     |████████████████████████████████| 2.9MB 25.1MB/s 
     |████████████████████████████████| 890kB 32.3MB/s 
     |████████████████████████████████| 1.1MB 36.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9f04890d337059e942138cc5e23e51e3a860dc4c6ea416e1e2a7aab5e25be2ad
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [ ]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [ ]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "My occupation is" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)

Predict next  3  words


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 7, 30522])
['teaching', 'unknown', 'business']
teaching    unknown    business
